In [ ]:
import os
import sys
import subprocess

# --- CONFIGURAZIONE DEL PROGETTO ---
# Modifica queste variabili per clonare il tuo repository specifico
GIT_REPO_URL = "https://github.com/Biobay/DeepLearning/"  # <-- INSERISCI QUI L'URL DEL TUO REPOSITORY
BRANCH_NAME = "RIS-1-CORRETTO"  # <-- INSERISCI QUI IL NOME DEL BRANCH
PROJECT_DIR = "DeepLearning" # <-- NOME DELLA CARTELLA DOPO LA CLONAZIONE

# --- SCRIPT DI SETUP E TRAINING ---

# 1. Clona il repository e si sposta nel branch specificato
if not os.path.exists(PROJECT_DIR):
    print(f"Clonazione del branch '{BRANCH_NAME}' da '{GIT_REPO_URL}'...")
    subprocess.run(['git', 'clone', '--branch', BRANCH_NAME, GIT_REPO_URL, PROJECT_DIR], check=True)
else:
    print(f"La cartella '{PROJECT_DIR}' esiste già. Salto la clonazione.")

# 2. Naviga nella cartella del progetto
os.chdir(PROJECT_DIR)
print(f"Cartella di lavoro corrente: {os.getcwd()}")

# 3. Installa le dipendenze
print("Installazione delle dipendenze da requirements.txt...")
subprocess.run([sys.executable, '-m', 'pip', 'install', '-r', 'requirements.txt'], check=True)

# 4. Aggiunge il percorso al path di Python per gli import
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
print("Path di sistema aggiornato.")

# 5. Avvia l'addestramento
print("--- Avvio del processo di addestramento ---")
try:
    from scripts.train import train
    import src.config as config
    print("Moduli del progetto importati con successo!")
    train(config)
    print("--- Processo di addestramento terminato ---")
except ImportError as e:
    print(f"ERRORE: Impossibile importare i moduli. {e}")
except Exception as e:
    print(f"Si è verificato un errore durante l'addestramento: {e}")


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import os

# --- VISUALIZZAZIONE DELLE LOSS ---

# 1. Definisci il percorso del file di log
# Assicurati che la cartella del progetto sia quella corrente
PROJECT_DIR = "DeepLearning"
LOG_FILE_PATH = os.path.join(PROJECT_DIR, "results", "logs", "loss_log.csv")

# 2. Controlla se il file di log esiste
if os.path.exists(LOG_FILE_PATH):
    print(f"Trovato file di log in: {LOG_FILE_PATH}")
    
    # 3. Carica i dati con pandas
    df_loss = pd.read_csv(LOG_FILE_PATH)
    
    # 4. Crea il grafico
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, ax1 = plt.subplots(figsize=(15, 7))

    # Plotta le loss principali (D e G)
    ax1.plot(df_loss.index, df_loss['loss_d'], label='Discriminator Loss (Total)', color='red', alpha=0.8)
    ax1.plot(df_loss.index, df_loss['loss_g'], label='Generator Loss (Total)', color='blue', alpha=0.8)
    
    # Plotta le componenti della loss del generatore
    ax1.plot(df_loss.index, df_loss['loss_g_adv'], '--', label='Generator Loss (Adversarial)', color='cyan', alpha=0.7)
    ax1.plot(df_loss.index, df_loss['loss_g_l1'], '--', label='Generator Loss (L1)', color='lightblue', alpha=0.7)

    # Impostazioni del grafico
    ax1.set_title('Andamento delle Loss di Generatore e Discriminatore (Stage-I)', fontsize=16)
    ax1.set_xlabel('Batch Number', fontsize=12)
    ax1.set_ylabel('Loss Value', fontsize=12)
    ax1.legend(loc='upper right')
    ax1.grid(True)
    
    # Migliora la leggibilità degli assi
    plt.xticks(rotation=45)
    plt.tight_layout()
    
    # 5. Salva il grafico
    plot_path = os.path.join(PROJECT_DIR, "results", "loss_plot.png")
    plt.savefig(plot_path)
    print(f"Grafico delle loss salvato in: {plot_path}")
    
    # Mostra il grafico
    plt.show()

else:
    print(f"ERRORE: File di log non trovato in '{LOG_FILE_PATH}'. Esegui prima la cella di training.")
